## AIT526-S01 Lab Assignment 
---------------------------------------------------------------------------------------
Created by Dr. Liao (6/5/2020) Professor: Maryam Heidari

---------------------------------------------------------------------------------------
#### Group 4
Students' Full Name: Nina Lin, Penny O'Brien, Maniphone Sourivong, Muge Yalcin

Submission Date: 7/1/2023

---------------------------------------------------------------------------------------
### Question Answering

##### Import libraries

In [1]:
import wikipedia # Module for retrieving information from Wikipedia
import sys # Module for system-specific parameters and functions
import re # Module for working with regular expressions
from datetime import datetime # Module for working with dates and times
from nltk.corpus import stopwords # Corpus of stop words for filtering text
from nltk import ne_chunk, pos_tag, word_tokenize # Functions for NLP tasks
from nltk.chunk import tree2conlltags # Function for converting parse tree to IOB-tagged tuples
import tkinter as tk # Module for creating GUI
import logging # Module for logging
import os ## Module for interacting with the operating system
import datetime

##### Set up logger

In [2]:
# Set up logging configuration
logging.basicConfig(
    filename='myfilelog.txt', # Log file name
    level=logging.INFO, # Logging level
    format='%(asctime)s - %(levelname)s - %(message)s' # Log message format
)
logger = logging.getLogger() # Logger object for writing logs

# Function to log the script execution details
def log_execution(script_path):
    # Get the directory of the script
    script_dir = os.path.dirname(script_path)

    # Create the log file path
    log_file_path = os.path.join(script_dir, "myfilelog.txt")

    # Get the current date and time
    current_time = datetime.datetime.now()

    # Create a log message for script start
    log_message = f"Script execution started on {current_time}\n"

    # Write the log message to the file
    with open(log_file_path, "a") as log_file:
        log_file.write(log_message)

    # Execute the script and capture stdout
    stdout_backup = sys.stdout
    with open(log_file_path, "a") as log_file:
        sys.stdout = log_file  # Redirect stdout to the log file

        try:
            # Execute the script
            main()
        except Exception as e:
            # Log any exceptions that occur during script execution
            error_message = f"Exception occurred: {str(e)}\n"
            sys.stderr.write(error_message)

    # Restore stdout
    sys.stdout = stdout_backup

    # Create a log message for script end
    current_time = datetime.datetime.now()
    log_message = f"Script execution ended on {current_time}\n"

    # Write the log message to the file
    with open(log_file_path, "a") as log_file:
        log_file.write(log_message)

    print("Script execution logged successfully.")

##### Set up main function and create GUI window

In [3]:
def main():
    # Get the current date and time
    current_time = datetime.datetime.now()
    logger.info(f"Script execution started on {current_time}")
    
    try:
        log = open(sys.argv[1], "w+")  # Open a file for writing logs
        counter = 1  # Initialize the counter for question number

        # Create GUI window
        window = tk.Tk()  # Create a Tkinter window
        window.title("Question Answering System")  # Set the title of the window
        window.geometry("500x300")  # Set the dimensions of the window
        window.configure(bg="#cffdbc")  # Set the background color

        # Create input box
        input_label = tk.Label(window, text="Enter a question:", bg="#cffdbc", font=("Arial", 10))
        input_label.pack()
        # Add note label
        note_label = tk.Label(window, text="(Please enter a valid question, or click exit to close the window)", bg="#cffdbc", font=("Arial", 10, "italic"))
        note_label.pack()
        input_entry = tk.Entry(window, width=50)
        input_entry.pack()

        # Create output box with scrollbar
        output_label = tk.Label(window, text="Answer:", bg="#cffdbc", font=("Arial", 10))
        output_label.pack()
        output_frame = tk.Frame(window)
        output_frame.pack(fill=tk.BOTH, expand=True)
        output_text = tk.Text(output_frame, height=10, width=50)
        output_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar = tk.Scrollbar(output_frame, orient=tk.VERTICAL, command=output_text.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        output_text.config(yscrollcommand=scrollbar.set)

        def process_question(event=None):  # event=None allows binding to work with or without event parameter
            nonlocal counter
            s = input_entry.get()  # Get the entered question
            e = s
            log.write(str(counter) + "Q) " + s + "\n")  # Write the question to the log file
            flag = False
            final = ""
            wiki = ""
            match = None  # Define match with a default value of None
            if s != "" and s is not None:  # Check if the question is not empty
                s = s.title()  # Convert the question to title case
                n = token(s)  # Tokenize and tag the question
                r = question(s)  # Identify the type of question
                if r != "":  # Check if a valid question type is identified
                    if "Where" in s or "What" in s:  # Check if the question is about location or general information
                        wiki = ans(r, log)  # Retrieve information from Wikipedia
                        match = re.search(r'\d{4}', wiki)  # Check for a year in the retrieved information
                        if match is not None:
                            wiki = ""  # If a year is found, clear the retrieved information
                    if 'B-PERSON' in n[2] or 'I-PERSON' in n[2]:  # Check if the question contains named entities of type PERSON
                        flag = True
                    elif 'I-PERSON' in n[2] or 'B-PERSON' in n[2]:  # Check if the question contains named entities of type PERSON
                        flag = True
                    if "Who" in s and flag:  # Check if the question is about a person
                        wiki = ans(r, log)  # Retrieve information from Wikipedia
                    if "When" in s or "Age" in s and flag:  # Check if the question is about a person's age or a specific year
                        wiki = ans(r, log)  # Retrieve information from Wikipedia
                        result = func(wiki)  # Extract relevant information from the retrieved data
                        if result != "":  # Check if relevant information is extracted
                            if "Age" in s:  # Check if the question is about age
                                match = re.search(r'\d{4}', result)  # Check for a year in the extracted information
                                if match is not None:
                                    result = datetime.now().year - int(match.group())  # Calculate the age
                                else:
                                    result = ""  # If no year is found, clear the result
                            appends = reg(s, result)  # Generate a formatted answer
                            if appends is not None:
                                final = appends
                            else:
                                final = ""  # If the answer format is not recognized, clear the result
                        else:
                            final = ""  # If no relevant information is extracted, clear the result
                    else:
                        final = wiki  # Use the retrieved information as the answer
                else:
                    final = ""  # If the question type is not identified, clear the result
            else:
                output_text.insert(tk.END, "Please ask a valid question!")  # Display a message for invalid question

            if final == "":  # Check if the final answer is empty
                log.write(str(counter) + "A) Answer not found.\n\n")  # Write "Answer not found" to the log file
                output_text.insert(tk.END, "I am sorry, I don't know the answer.\n")  # Display "Answer not found" message
            else:
                try:
                    log.write(str(counter) + "A) " + final + "\n\n")  # Write the answer to the log file
                    output_text.insert(tk.END, final + "\n\n")  # Display the answer in the output box
                except Exception as GeneralException:
                    log.write(str(counter) + "A) Answer not found.\n\n")  # Write "Answer not found" to the log file
                    output_text.insert(tk.END, "I am sorry, I don't know the answer.\n\n")  # Display "Answer not found" message
            counter = counter + 1  # Increment the question counter
            input_entry.delete(0, tk.END)  # Clear the input box
            output_text.see(tk.END)  # Scroll to the end of the output box

        # Create submit button
        submit_button = tk.Button(window, text="Submit", command=process_question)  # Create a button to submit the question
        submit_button.pack()

        # Bind Enter key press event to submit function
        window.bind("<Return>", process_question)  # Bind the Enter key to the submit function

        window.mainloop()  # Start the GUI event loop

    except Exception as GeneralException:
        print(GeneralException)  # Print any general exception that occurs
    finally:
        log.close()  # Close the log file

##### Define question patterns and retrieve answers

In [ ]:
def question(s):
    x = ""  # Initialize the variable for storing the identified question type
    stats = [
        [r'When (Is|Was) (.+) Born', ["{0}"]],  # Regex pattern for questions about birthdate
        [r'When (Is|Was) (.+) Birthday', ["{0}"]],  # Regex pattern for questions about birthday
        [r'Where (Is|Was) (.+)', ["{0}"]],  # Regex pattern for questions about location
        [r'Who (Is|Was) (.+)', ["{0}"]],  # Regex pattern for questions about a person
        [r'What (Is|Was) (.+) Age', ["{0}"]],  # Regex pattern for questions about age
        [r'What (Is|Was) (.+)', ["{0}"]],  # Regex pattern for general questions
    ]
    for r, u in stats:
        m = re.match(r, s.rstrip('.!?'))  # Match the question against each pattern
        if m:
            x = m.groups()[1]  # Extract the relevant part of the question
            break
    return x  # Return the identified question type

def token(u):
    tokens = word_tokenize(u)  # Tokenize the input question
    l = set(stopwords.words('english'))  # Get the set of stopwords
    l.add('?')  # Add the question mark to stopwords
    string = []
    for i in tokens:
        if i not in l:  # Filter out stopwords
            string.append(i)
    t = tree2conlltags(ne_chunk(pos_tag(word_tokenize(u))))  # Perform named entity recognition
    return t  # Return the tagged tokens

def ans(query, log):
    try:
        queryResult = wikipedia.summary(query, sentences=1)  # Retrieve a summary from Wikipedia
    except Exception as GeneralException:
        queryResult = ""  # If an exception occurs, clear the result
    return queryResult  # Return the retrieved information


def func(s):
    m = ""  # Initialize the variable for storing the extracted information
    stats = [
        [r'[\d]+\s(\w)+\s[\d]+', ["{0}"]],  # Regex pattern for extracting a specific date format
        [r'(?<=\().(\w+).(\w+,).(\w+)', ["{0}"]],  # Regex pattern for extracting a specific date format within parentheses
    ]
    for r, u in stats:
        m1 = re.search(r, s)  # Search for the pattern in the input string
        if m1 is not None:
            m = m1.group()  # Extract the relevant part of the string
            break
    return m  # Return the extracted information

def reg(u, q):
    u = u.rstrip('.!')  # Remove trailing punctuation from the question
    ur = re.sub(r'When (Is|Was) (.+) Born', r'\2 Was Born On ', u)  # Replace the question format for birthdate
    if ur != u:
        return ur + str(q) + '.'  # Return the formatted answer
    ur = re.sub(r'When (Is|Was) (.+) Birthday', r'\2 Birthday Is On ', u)  # Replace the question format for birthday
    if ur != u:
        ur = ur + q
        ur = re.sub(r'\d{4}', '', ur)  # Remove the year from the answer
        return ur + '.'  # Return the formatted answer
    ur = re.sub(r'Where (Is|Was) (.+)', r'\2 Is Located In ', u)  # Replace the question format for location
    if ur != u:
        return ur + str(q) + '.'  # Return the formatted answer
    ur = re.sub(r'Who (Is|Was) (.+)', r'\2 Is ', u)  # Replace the question format for a person
    if ur != u:
        return ur + str(q) + '.'  # Return the formatted answer
    ur = re.sub(r'What (Is|Was) (.+) Age', r'\2 Is ', u)  # Replace the question format for age
    if ur != u:
        return ur + str(q) + '.'  # Return the formatted answer
    ur = re.sub(r'What (Is|Was) (.+)', r'\2 Is ', u)  # Replace the question format for general questions
    if ur != u:
        return ur + str(q) + '.'  # Return the formatted answer
    return None  # Return None if the answer format is not recognized

    # Get the current date and time
    current_time = datetime.datetime.now()
    logger.info(f"Script execution ended on {current_time}")

if __name__ == '__main__':
    log_execution(sys.argv[0])  # Call the log_execution function to log the script execution

##### Reference: Dr. Liao’s Python code Examples & tutorials
All python code referenced are from Dr. Liao unless otherwise stated under the specific code.